# Config

In [1]:
from delta import DeltaTable
from datetime import datetime

import ConnectionConfig as cc
cc.setupEnvironment()

# Start local cluster

In [2]:
spark = cc.startLocalCluster("DIM_LOCK")
spark.getActiveSession()

# Ophalen van gegevens (en opslaan in data warehouse)

In [3]:
#Extract
cc.set_connectionProfile("VeloDB")

df_lock = spark.read \
    .format("jdbc") \
    .option("driver" , cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "locks") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "lockid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 8000) \
    .load()

df_station = spark.read \
    .format("jdbc") \
    .option("driver" , cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "stations") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "stationid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 350) \
    .load()

df_lock.show(5)
df_station.show(5)

+------+-------------+---------+---------+
|lockid|stationlocknr|stationid|vehicleid|
+------+-------------+---------+---------+
|     1|            1|        1|     NULL|
|     2|            2|        1|     NULL|
|     3|            3|        1|     NULL|
|     4|            4|        1|     NULL|
|     5|            5|        1|     NULL|
+------+-------------+---------+---------+
only showing top 5 rows

+---------+--------+---------+------------+--------------------+------+-------+---------+-----------------+-------------------+-------+------+
|stationid|objectid|stationnr|        type|              street|number|zipcode| district|         gpscoord|     additionalinfo|labelid|cityid|
+---------+--------+---------+------------+--------------------+------+-------+---------+-----------------+-------------------+-------+------+
|        1|   33202|      026|DUBBELZIJDIG|         Meir (2000)|    84|   2000|ANTWERPEN|(51.2182,4.41241)|                   |   NULL|  NULL|
|        2|   33

In [25]:
#Transform
df_lock.createOrReplaceTempView("source_Lock")
df_station.createOrReplaceTempView("source_Station")

#querien van de data
df_dim_lock= spark.sql(f"select uuid() as lock_SK, lock.lockid, lock.stationLockNr, station.stationid, station.stationNr, station.type,\
                        station.street, station.number, station.district\
                        from source_Lock as lock\
                        join source_Station as station on station.stationid = lock.stationid")

df_dim_lock.show()

+--------------------+------+-------------+---------+---------+-----------+--------------------+------+---------+
|             lock_SK|lockid|stationLockNr|stationid|stationNr|       type|              street|number| district|
+--------------------+------+-------------+---------+---------+-----------+--------------------+------+---------+
|5a53f58f-3f85-439...|   234|           18|       12|      120|ENKELZIJDIG|Schijnpoortweg (2...| 27-29|ANTWERPEN|
|1f219141-c17a-413...|   233|           17|       12|      120|ENKELZIJDIG|Schijnpoortweg (2...| 27-29|ANTWERPEN|
|5c941285-64aa-47e...|   232|           16|       12|      120|ENKELZIJDIG|Schijnpoortweg (2...| 27-29|ANTWERPEN|
|b0f481e8-3f52-42a...|   231|           15|       12|      120|ENKELZIJDIG|Schijnpoortweg (2...| 27-29|ANTWERPEN|
|4d386bfd-def4-40f...|   230|           14|       12|      120|ENKELZIJDIG|Schijnpoortweg (2...| 27-29|ANTWERPEN|
|015c475c-fc06-4a8...|   229|           13|       12|      120|ENKELZIJDIG|Schijnpoortwe

In [29]:
df_dim_lock.insertInto("'test', null, null, null, null, 'geen slot', null, null, null")

AttributeError: 'DataFrame' object has no attribute 'insertInto'

In [28]:
spark.sql(f"insert into df_dim_lock \
          (lock_SK,lockid, stationLockNr, stationid, stationNr, type, street, number, district)\
          values('test', null, null, null, null, 'geen slot', null, null, null)")

AttributeError: 'DataFrame' object has no attribute 'insertInto'